In [19]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import re
import ast

In [20]:
# Read the CSV files
listings = pd.read_csv("listings.csv")
listings_scrap = pd.read_csv("listings_scrapy.csv")

# Merge the two dataframes on the 'href' column
merged_df = pd.merge(listings, listings_scrap, on='href', how='inner')

In [21]:
# Convert string representations of dictionaries to dictionaries
merged_df['main_features'] = merged_df['main_features'].apply(ast.literal_eval)

# Extract numeric values and convert to integers
merged_df['Floor Area'] = merged_df['main_features'].apply(lambda x: int(re.sub(r'\D', '', x.get('Floor Area', '0'))))
merged_df['Land Area'] = merged_df['main_features'].apply(lambda x: int(re.sub(r'\D', '', x.get('Land Area', '0'))))
merged_df['Rates'] = merged_df['main_features'].apply(lambda x: int(re.sub(r'\D', '', x.get('Rates', '0'))))
merged_df['Levy'] = merged_df['main_features'].apply(lambda x: int(re.sub(r'\D', '', x.get('Levy', '0'))))

In [22]:
merged_df['key_main_features'] = merged_df['key_main_features'].apply(ast.literal_eval)

# Extracting numeric values for Bedrooms and Bathrooms and converting to float
merged_df['Bedrooms'] = merged_df['key_main_features'].apply(lambda x: float(re.search(r'\d+(\.\d+)?', x.get('Bedrooms', '0')).group()))
merged_df['Bathrooms'] = merged_df['key_main_features'].apply(lambda x: float(re.search(r'\d+(\.\d+)?', x.get('Bathrooms', '0')).group()))

In [25]:
merged_df['Province'] = merged_df['href'].apply(lambda x: x.split('/')[4])
merged_df['City'] = merged_df['href'].apply(lambda x: x.split('/')[5])
merged_df['Suburb'] = merged_df['href'].apply(lambda x: x.split('/')[6])

# Conditionally splitting the 'Area' column
merged_df['Area'] = merged_df.apply(lambda row: row['href'].split('/')[7] if 'midrand' not in row['href'] else row['href'].split('/')[6], axis=1)

# Extracting the listing number from the URL
merged_df['Listing Number'] = merged_df['href'].apply(lambda x: x.split('/')[-1])


In [26]:
merged_df.head()

,title,href,price,property_type,main_features,key_main_features,Floor Area,Land Area,Rates,Levy,Bedrooms,Bathrooms,Province,City,Suburb,Area,Listing Number
0,3 Bed House in Rynfield,https://www.privateproperty.co.za/for-sale/gau...,R 3 390 000,House,"{'Floor Area': '310 m²', 'Land Area': '570 m²'...","{'Bedrooms': '3', 'Bathrooms': '2.5', 'Dining ...",310,570,1930,1450,3.0,2.5,gauteng,east-rand,benoni,rynfield,T11453
1,3 Bed House in Lakefield,https://www.privateproperty.co.za/for-sale/gau...,R 3 490 000,House,"{'Floor Area': '309 m²', 'Rates': 'R 2104', 'L...","{'Bedrooms': '3', 'Bathrooms': '3.5', 'Dining ...",309,0,2104,650,3.0,3.5,gauteng,east-rand,benoni,lakefield,T4042182
2,3 Bed House in Lakefield,https://www.privateproperty.co.za/for-sale/gau...,R 5 300 000,House,"{'Land Area': '1405 m²', 'Rates': 'R 3655', 'L...","{'Bedrooms': '3', 'Bathrooms': '3.5', 'Covered...",0,1405,3655,650,3.0,3.5,gauteng,east-rand,benoni,lakefield,T4042186
3,4 Bed House in Rynfield,https://www.privateproperty.co.za/for-sale/gau...,R 6 000 000,House,"{'Floor Area': '498 m²', 'Land Area': '2468 m²...","{'Bedrooms': '4', 'Bathrooms': '3.5', 'Garages...",498,2468,3000,1570,4.0,3.5,gauteng,east-rand,benoni,rynfield,T4362174
4,4 Bed House in Ebotse Estate,https://www.privateproperty.co.za/for-sale/gau...,R 11 800 000,House,"{'Floor Area': '724 m²', 'Land Area': '838 m²'...","{'Bedrooms': '4', 'Bathrooms': '4.5', 'Dining ...",724,838,5682,3800,4.0,4.5,gauteng,east-rand,benoni,ebotse-estate,T4478534


In [29]:
sold_properties = merged_df[merged_df['price']=='Sold']
# Adding Sold Date column with today's date
sold_properties['Sold Date'] = datetime.today().date()

# Define the file path
file_path = r'https://github.com/Nikhil-Lakha/HouseFlipping/tree/main/Sold%20Properties'

# Generate the file name with today's date
file_name = "Sold Properties " + datetime.today().strftime('%Y-%m-%d') + ".csv"

# Combine the file path and file name
full_file_path = file_path + "//" + file_name

# Save the DataFrame to CSV
sold_properties.to_csv(full_file_path, index=False)

C:\Users\lakha\AppData\Local\Temp\ipykernel_8596\2732487550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sold_properties['Sold Date'] = datetime.today().date()


InvalidURL: URL can't contain control characters. '/Nikhil-Lakha/HouseFlipping/tree/main/Sold%20Properties//Sold Properties 2024-03-10.csv' (found at least ' ')

In [ ]:
listed_properties = merged_df[(merged_df['price'] != 'Sold') & (merged_df['price'] != 'On Auction')]
# Remove non-numeric characters and convert to numeric
listed_properties['price'] = listed_properties['price'].str.replace('[^/d.]', '', regex=True)

# Convert the column to numeric
listed_properties['price'] = pd.to_numeric(listed_properties['price'], errors='coerce')
# Define the directory path

listed_properties['Price per sqm'] = listed_properties['price']/listed_properties['Floor Area']
directory_path = r'https://github.com/Nikhil-Lakha/HouseFlipping/tree/main/Listed%20Properties'

# Save as "Current Listed Properties.csv"
listed_properties.to_csv(directory_path + "//Current Listed Properties.csv", index=False)

# Save with today's date
today_date = datetime.today().strftime('%Y-%m-%d')
file_name = f"Listed Properties {today_date}.csv"
listed_properties.to_csv(directory_path + "//" + file_name, index=False)

## Market Research

# Past Listed Properties

In [ ]:
directory = "https://github.com/Nikhil-Lakha/HouseFlipping/tree/main/Listed%20Properties"

In [ ]:
# Initialize an empty DataFrame to store the data
listed_combined_data = pd.DataFrame()

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("Listed Properties"):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        data = pd.read_csv(file_path)
        
        # Append the data to the combined DataFrame
        listed_combined_data = pd.concat([listed_combined_data, data], ignore_index=True)
        
# Remove duplicates based on the 'Listing Number' column, keeping only the oldest 'Sold Date'
listed_combined_data.drop_duplicates(subset='Listing Number', keep='first', inplace=True)
listed_combined_data.to_csv('https://raw.githubusercontent.com/Nikhil-Lakha/HouseFlipping/main/Listed%20Properties/Past%20Listed%20Properties.csv',index=False)

# All Sold Properties

In [ ]:
# Define the directory where the CSV files are located
directory = 'https://github.com/Nikhil-Lakha/HouseFlipping/tree/main/Sold%20Properties'

In [ ]:
# Initialize an empty DataFrame to store the data
combined_data = pd.DataFrame()

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("Sold Properties"):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        data = pd.read_csv(file_path)
        
        # Convert 'Sold Date' column to datetime
        data['Sold Date'] = pd.to_datetime(data['Sold Date'])
        
        # Sort the data by 'Sold Date' in ascending order
        data.sort_values(by='Sold Date', inplace=True)
        
        # Append the data to the combined DataFrame
        combined_data = pd.concat([combined_data, data], ignore_index=True)
        
# Remove duplicates based on the 'Listing Number' column, keeping only the oldest 'Sold Date'
combined_data.drop_duplicates(subset='Listing Number', keep='first', inplace=True)

combined_data = pd.merge(combined_data, listed_combined_data[['Listing Number', 'price']], on='Listing Number', how='left')
# Rename the 'price' column to 'previously listed price'
combined_data.rename(columns={'price_y': 'previously listed price'}, inplace=True)
combined_data.rename(columns={'price_x': 'Status'}, inplace=True)

In [ ]:
combined_data.to_csv('https://raw.githubusercontent.com/Nikhil-Lakha/HouseFlipping/main/Sold%20Properties/All%20Sold%20Properties.csv',index=False)

# Comparing Current listed with Previously 30 Days Sold

In [ ]:
current_listed = data = pd.read_csv('https://raw.githubusercontent.com/Nikhil-Lakha/HouseFlipping/main/Listed%20Properties/Current%20Listed%20Properties.csv')

In [ ]:
# Get the date 30 days ago from today
thirty_days_ago = datetime.now() - timedelta(days=30)

# Filter the DataFrame to include only rows where sale date is within the last 30 days
days_sold_data = combined_data[combined_data['Sold Date'] >= thirty_days_ago]

In [ ]:
listed_property_count = listed_combined_data.groupby(['Province', 'City', 'Suburb', 'Area']).size().reset_index(name='Listed')
sold_property_count = days_sold_data.groupby(['Province', 'City', 'Suburb', 'Area']).size().reset_index(name='Sold')

In [ ]:
# Merging two dataframes on the 'Area' column and excluding common columns
# Merging two dataframes on the 'Area' column
merged_df = pd.merge(listed_property_count, sold_property_count, on='Area', how='left', suffixes=('_listed', '_sold'))

# Drop unwanted columns (those with '_y' suffix)
merged_df.drop(merged_df.filter(regex='_sold$').columns.tolist(), axis=1, inplace=True)

# Rename columns with '_x' suffix to remove the suffix
merged_df.rename(columns=lambda x: x.replace('_listed', ''), inplace=True)
merged_df['Inventory'] = merged_df['Listed']/merged_df['Sold']
merged_df.to_csv('Property Market 30 days.csv', index=False)